### Imports

In [2]:
import pandas as pd
import cbpro
import datetime
from datetime import datetime as dt
import numpy as np
import time
import pickle
import configparser

In [29]:
## Bring in cbpro public client
public_client = cbpro.PublicClient()
products = public_client.get_products()

In [30]:
product_names = []
for i in products:
    product_names.append(i['id'])

In [31]:
config = configparser.ConfigParser()
config.read(r'C:\Users\hlu\repos\coinbase_v2\config_global.ini')

['C:\\Users\\hlu\\repos\\coinbase_v2\\config_global.ini']

### Parameters 

In [32]:
now = datetime.datetime.now()
end = now - datetime.timedelta(days=1)
start = end - datetime.timedelta(days=100)
candles = 200 # Max number of candles that can be ingested through API
instruments = product_names
granularity = config['PARAMETERS']['granularity']
if granularity =="minutes":
    granularity = 60

### Functions

In [33]:
#Function for pulling in data fromcbpro
def pull_data(gran_,start,end,candles,instrument):    #granularity [60, 300, 900, 3600, 21600, 86400]
    accepted_gran = [60, 300, 900, 3600, 21600, 86400]
    if gran_ not in accepted_gran:
        print("granularity wrong")
        return
    gran_mins = gran_/60
    gran_hours = gran_mins/60
    gran_days = gran_hours / 24
    duration = ((end-start)/gran_).seconds + ((end-start)/gran_).days*24*60*60
    print(duration)
    iterations = int(np.ceil(duration/200))
    new_df = pd.DataFrame()
    print("starting")
    print(iterations)
    for i in range(0,iterations):
        #print(i)
        time.sleep(0.2)
        if i ==0:
            end = end
            start = end-datetime.timedelta(minutes = gran_mins*200)
            v = public_client.get_product_historic_rates(instrument, start, end,  granularity=gran_)
            BTC_prices = pd.DataFrame(v)
            new_df = new_df.append(BTC_prices)
        else:
            end = end-datetime.timedelta(minutes = gran_mins*201)
            start = end-datetime.timedelta(minutes = gran_mins*200)
            v = public_client.get_product_historic_rates(instrument, start, end,  granularity=gran_)
            BTC_prices = pd.DataFrame(v)
            new_df = new_df.append(BTC_prices)
    print(new_df)
    new_df['time'] = new_df[0].apply(lambda x: dt.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
    new_df = new_df.iloc[::-1].reset_index(drop=True)
    new_df = new_df.drop(columns =[0])
    new_df = new_df.rename(columns = {
                                            1:"low"
                                            ,2:"high"
                                            ,3:"open"
                                            ,4:"close"
                                            ,5:"volume"
                                            })
    return new_df

### Pull data

In [40]:
for item in instruments[25:] :
    print(item)
    data = pull_data(granularity,start,end,candles,item)
    string_ = "minute_last_100_days_"+str(item)
    data.to_csv(string_+".csv")


BAT-ETH
144000
starting
720
             0         1         2         3         4       5
0   1612999320  0.000253  0.000255  0.000254  0.000254   18250
1   1612999200  0.000253  0.000254  0.000254  0.000254  114539
2   1612999140  0.000254  0.000254  0.000254  0.000254    6537
3   1612999080  0.000254  0.000254  0.000254  0.000254    3310
4   1612998960  0.000252  0.000253  0.000252  0.000253      44
..         ...       ...       ...       ...       ...     ...
16  1604318160  0.000479  0.000480  0.000479  0.000480      16
17  1604317560  0.000479  0.000479  0.000479  0.000479     150
18  1604317380  0.000479  0.000479  0.000479  0.000479     301
19  1604316960  0.000480  0.000480  0.000480  0.000480     432
20  1604316540  0.000480  0.000480  0.000480  0.000480     180

[35376 rows x 6 columns]
UMA-EUR
144000
starting
720
             0       1       2       3       4         5
0   1612999440  23.852  23.987  23.941  23.852  1256.015
1   1612999380  23.940  23.967  23.940  23.948  

ReadTimeout: HTTPSConnectionPool(host='api.pro.coinbase.com', port=443): Read timed out. (read timeout=30)